In [1]:
from word2vec import *

In [2]:
words = ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
vocab = set(words)
embedding_size = 10

word2vec = Word2Vec(vocab, embedding_size)

## Backward
This is the forward graph
```
embedding --(@ linear)--> x --(softmax)--> o --(criterion)--> loss
```

Gradient descent step

In [3]:
input_index, output_index = 4, 1

for _ in range(20):
    word2vec.step([input_index], output_index)
_, output = word2vec.forward([input_index])
output

Loss - 1.502
Loss - 1.017
Loss - 0.640
Loss - 0.333
Loss - 0.121
Loss - 0.044
Loss - 0.025
Loss - 0.017
Loss - 0.013
Loss - 0.011
Loss - 0.009
Loss - 0.008
Loss - 0.007
Loss - 0.006
Loss - 0.005
Loss - 0.005
Loss - 0.004
Loss - 0.004
Loss - 0.004
Loss - 0.003


array([0.00118292, 0.97423206, 0.0012486 , 0.00118962, 0.00334013,
       0.0024051 , 0.01362874, 0.00277284])

It learns very well!